In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()  # Ensure you have the correct driver for the browser
driver.get('https://mit.s.dk/studiebolig/login/')

username = driver.find_element(By.ID, 'id_username')
password = driver.find_element(By.ID,'id_password')
submit = driver.find_element(By.ID, 'id_login')

# Enter your login information and submit
username.send_keys(' yourusername')
password.send_keys(' yourpasscode')
submit.click()

driver.get('https://mit.s.dk/studiebolig/home/')

link = driver.find_element(By.CSS_SELECTOR, 'a[href="#collapse-10"]')
link.click()

In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

def click_until_gone(xpath):
    try:
        while True:
            try:
                button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, xpath))
                )
                button.click()
                WebDriverWait(driver, 5).until(
                    EC.staleness_of(button)
                )
            except TimeoutException:
                if driver.find_elements(By.XPATH, xpath):
                    continue  # Button still there, loop will continue
                print("Button no longer appears or is not clickable. Exiting loop.")
                break  # Button gone, exit loop
    except NoSuchElementException:
        print("Button was never present.")

button_xpath = '//*[@id="collapse-10"]/div/div/div[2]/div/button'

click_until_gone(button_xpath)


In [ ]:
# Find all the <a> tags in the specified div

links = driver.find_elements(By.CSS_SELECTOR, "#collapse-10 > div > div > div.col-12.col-md-6.order-md-1 > div > a")
hrefs = []
for link in links:
    href = link.get_attribute('href')
    print(href)
    hrefs.append(href)

In [ ]:
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

df = pd.DataFrame(columns=['name', 'address', 'ranking', 'size & price', 'url'])

for h in hrefs:
    driver.get(h)
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, 'body > main > form > header > h1'))
    )
    
    # Initialize variables as None
    name = None
    address = None
    ranking = None

    # Attempt to get the name
    try:
        name = driver.find_element(By.CSS_SELECTOR, 'body > main > form > header > h1').text
    except NoSuchElementException:
        print("Name element not found for URL:", h)

    try:
        price = driver.find_element(By.CSS_SELECTOR, "#heading-1 > h3 > a > span > span > span").get_attribute('innerText')
    except NoSuchElementException:
        print("Name element not found for URL:", h)

    # Attempt to get the address
    try:
        address = driver.find_element(By.CSS_SELECTOR, "#collapse-1 > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > a").get_attribute('text')
    except NoSuchElementException:
        print("Address element not found for URL:", h)

    # Attempt to get the ranking
    try:
        ranking = driver.find_element(By.CSS_SELECTOR, '#collapse-1 > div > table > tbody > tr:nth-child(1) > td:nth-child(3) > a > span.waiting-list-category').get_attribute('innerText')
        ranking = ranking.replace(" ","")[1] if ranking else None
    except NoSuchElementException:
        print("Ranking element not found for URL:", h)

    # Append the results to the DataFrame
    df.loc[len(df.index)] = [name, address, ranking, price, h]

driver.close()

In [ ]:
df

In [ ]:
df.to_csv('sdk.csv')